In [13]:
import os
import pandas as pd
import folium
from geopy.geocoders import Photon, Nominatim, GoogleV3
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

from dotenv import load_dotenv

In [15]:
# Load environment variables from .env file
load_dotenv()

# Read the API key from the environment variable
google_api_key = os.getenv('GOOGLE_API_KEY')

In [2]:
df = pd.read_csv("./buildings_annarbor.csv")

In [3]:
# Combine address components into a single address string
df['Address'] = df['Number'].astype(str) + ' ' + df['Street Name'] + ', ' + df['city'] + ', ' + df['state']

In [4]:
# df = df[df.Index == 42]
# df

In [5]:

# Initialize geolocator
# Replace 'YOUR_API_KEY' with your actual Google Maps API key

geolocator_google = GoogleV3(api_key=google_api_key)
geolocator_photon = Photon(timeout=10)
geolocator_nominatim = Nominatim(user_agent="my_app")

In [6]:
# Function to geocode an address
def geocode_address(address):
    try:
        # Attempt geocoding with google first        
        location = geolocator_google.geocode(address)
        if location:
            return location.latitude, location.longitude
        #Photon
        location = geolocator_photon.geocode(address)
        if location:
            return location.latitude, location.longitude
        
        # If Photon fails, try Nominatim as a fallback
        location = geolocator_nominatim.geocode(address)
        if location:
            return location.latitude, location.longitude
        
        # If both fail, return None or handle as needed
        return None
    
    except (GeocoderTimedOut, GeocoderUnavailable) as e:
        # Handle specific geocoding errors
        print(f"Geocoding error for address {address}: {e}")
        return None
    except Exception as e:
        # Handle any unexpected errors
        print(f"Unexpected error for address {address}: {e}")
        return None

# Example usage with pandas DataFrame
df['Coordinates'] = df['Address'].apply(geocode_address)

In [7]:
df[['Latitude', 'Longitude']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)

In [8]:
df[df.Latitude<0]

,Index,Number,Street Name,Details,Region,city,state,Address,Coordinates,Latitude,Longitude


In [9]:
df = df.reset_index()

In [10]:
df['Latitude']

0      42.282549
1      42.282346
2      42.282541
3      42.282345
4      42.282516
         ...    
479    42.263718
480    42.259603
481    42.277585
482    42.278057
483    42.265869
Name: Latitude, Length: 484, dtype: float64

In [11]:
# Initialize map, preview the results
m = folium.Map(location=[df['Latitude'][0], df['Longitude'][0]], zoom_start=15)

# Add markers to the map
for i, row in df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']],
                  popup=row['Address']).add_to(m)

# Save the map to an HTML file
# m.save('map.html')

# Display the map
m

In [12]:
df.to_csv('buildings_annarbor_coordinates.csv', index=False)